## Transformer 아키텍처 구현 실습

#### 0. prelims

In [3]:
# numpy 버전 확인
print(np.__version__)

1.26.4


In [8]:
# numpy 버전 다운그레이드
!pip install "numpy<2" --user

     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------------ ------------------------- 20.5/61.0 kB 330.3 kB/s eta 0:00:01
     ------------------------------- ------ 51.2/61.0 kB 435.7 kB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 540.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB 2.0 MB/s eta 0:00:08
   ---------------------------------------- 0.2/15.8 MB 2.0 MB/s eta 0:00:08
    --------------------------------------- 0.3/15.8 MB 2.4 MB/s eta 0:00:07
   - -------------------------------------- 0.5/15.8 MB 2.8 MB/s eta 0:00:06
   - -------------------------------------- 0.5/15.8 MB 2.6 MB/s eta 0:00:06
   - -------------------------------------- 0.6/15.8 MB 2.4 MB/s eta 0:00:07
   - -------------------------------------- 0.7/15.8 MB 2.1 MB/s eta 0:00:08
   - -------------------------------------- 0.7/15.8 MB 2.0 MB/s eta 0:00:08
  

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
blis 1.0.1 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")
%matplotlib inline

#### 1. Standard Encoder-decoder model

In [ ]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many 
    other models.
    """
    def __init__(self, encoder, decoder, input_embed, target_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.input_embed = input_embed
        self.target_embed = target_embed
        self.generator = generator
        
    def forward(self, input, target, input_mask, target_mask):
        "Take in and process masked input and target sequences."
        return self.decode(self.encode(input, input_mask), input_mask,
                            target, target_mask)
    
    def encode(self, input, input_mask):
        return self.encoder(self.input_embed(input), input_mask)
    
    def decode(self, memory, input_mask, target, target_mask):
        return self.decoder(self.target_embed(target), memory, input_mask, target_mask)

In [ ]:
class Generator(nn.Module):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)